In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs
th.backends.cudnn.benchmark = True

In [ ]:
from reversible2.util import set_random_seeds
cuda=True
set_random_seeds(2019011641, cuda)

rand_inputs = th.randn(100,128,1024,1, device='cuda')
print((rand_inputs.numel()*  4) / (1024 ** 2))

In [ ]:
def conv_add_block_3x3(n_c, n_i_c):
    return AdditiveBlock(
        nn.Sequential(
            nn.Conv2d(n_c // 2, n_i_c, (3, 1), stride=1, padding=(1, 0),
                      bias=True),
            nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2, (3, 1), stride=1, padding=(1, 0),
                      bias=True)),

        nn.Sequential(
            nn.Conv2d(n_c // 2, n_i_c, (3, 1), stride=1, padding=(1, 0),
                      bias=True),
            nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2, (3, 1), stride=1, padding=(1, 0),
                      bias=True)),
        switched_order=False)


def dense_add_block(n_c, n_i_c):
    return AdditiveBlock(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, ),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2, )),

        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c, ),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2, )),
        switched_order=False)

cuda = True
from reversible2.graph import Node
from reversible2.branching import CatChans, ChunkChans, Select
from copy import deepcopy
from reversible2.graph import Node
from reversible2.distribution import TwoClassDist

from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter

set_random_seeds(2019011641, cuda)
n_chans = 128
base_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),# 2 x 256
    conv_add_block_3x3(2*n_chans,32),
    conv_add_block_3x3(2*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 4 x 128
    conv_add_block_3x3(4*n_chans,32),
    conv_add_block_3x3(4*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 8 x 64
    conv_add_block_3x3(8*n_chans,32),
    conv_add_block_3x3(8*n_chans,32))
base_model.cuda();

branch_1_a =  nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False), # 8 x 32
    conv_add_block_3x3(8*n_chans,32),
    conv_add_block_3x3(8*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 16 x 16
    conv_add_block_3x3(16*n_chans,32),
    conv_add_block_3x3(16*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 32 x 8
    conv_add_block_3x3(32*n_chans,32),
    conv_add_block_3x3(32*n_chans,32),
)
branch_1_b = nn.Sequential(
    *(list(deepcopy(branch_1_a).children()) + [
    ViewAs((-1, 32*n_chans,16,1), (-1,512*n_chans)),
    dense_add_block(512*n_chans,32),
    dense_add_block(512*n_chans,32),
    dense_add_block(512*n_chans,32),
    dense_add_block(512*n_chans,32),
]))
branch_1_a.cuda();
branch_1_b.cuda();

branch_2_a = nn.Sequential(
    SubsampleSplitter(stride=[2,1], chunk_chans_first=False), # 32 x 4
    conv_add_block_3x3(32*n_chans,32),
    conv_add_block_3x3(32*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 64 x 2
    conv_add_block_3x3(64*n_chans,32),
    conv_add_block_3x3(64*n_chans,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 128 x 1
    ViewAs((-1, 128*n_chans,2,1), (-1,256*n_chans)),
    dense_add_block(256*n_chans,64),
    dense_add_block(256*n_chans,64),
    dense_add_block(256*n_chans,64),
    dense_add_block(256*n_chans,64),
)


branch_2_b = deepcopy(branch_2_a).cuda()
branch_2_a.cuda();
branch_2_b.cuda();

final_model = nn.Sequential(
    dense_add_block(1024*n_chans,256),
    dense_add_block(1024*n_chans,256),
    dense_add_block(1024*n_chans,256),
    dense_add_block(1024*n_chans,256),
    RFFT()
)
final_model.cuda();
o = Node(None, base_model)
o = Node(o, ChunkChans(2))
o1a = Node(o, Select(0))
o1b = Node(o, Select(1))
o1a = Node(o1a, branch_1_a)
o1b = Node(o1b, branch_1_b)
o2 = Node(o1a, ChunkChans(2))
o2a = Node(o2, Select(0))
o2b = Node(o2, Select(1))
o2a = Node(o2a, branch_2_a)
o2b = Node(o2b, branch_2_b)
o = Node([o1b,o2a,o2b], CatChans())
o = Node(o, final_model)
feature_model = o
if cuda:
    feature_model.cuda()
feature_model.eval();

In [ ]:
outs = feature_model(rand_inputs[:20])
inverted = feature_model.invert(outs)
loss = th.sum(inverted ** 2)
feature_model.zero_grad()
loss.backward()

In [ ]:
from reversible2.constantmemory import graph_to_constant_memory


In [ ]:
from reversible2.constantmemory import AdditiveBlockConstantMemory

cuda = True
from reversible2.graph import Node
from reversible2.branching import CatChans, ChunkChans, Select
from reversible2.graph import Node
from reversible2.wrap_invertible import WrapInvertible

from copy import deepcopy
from reversible2.graph import Node

from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter

set_random_seeds(2019011641, cuda)
n_chans = 128
feature_model2 = graph_to_constant_memory(deepcopy(feature_model))
if cuda:
    feature_model2.cuda()
feature_model2.eval();

for p1,p2 in zip(feature_model.parameters(), feature_model2.parameters()):
    assert p1 is not p2

In [ ]:
from reversible2.constantmemory import clear_ctx_dicts
outs2 = feature_model2(rand_inputs[:20])
inverted2 = feature_model2.invert(outs2)
loss2 = th.sum(inverted2 ** 2)
feature_model2.zero_grad()
loss2.backward()
assert th.allclose(outs, outs2)
assert th.allclose(inverted, inverted2)
assert th.allclose(loss, loss2)
for p1,p2 in zip(feature_model.parameters(), feature_model2.parameters()):
    assert p1 is not p2
    assert th.allclose(p1,p2)
    if not th.allclose(p1.grad, p2.grad):
        print("Mean: {:.0E}".format(th.mean(th.abs(p1.grad - p2.grad)).item()))
        print("Max:  {:.0E}".format(th.max(th.abs(p1.grad - p2.grad)).item()))
    assert th.allclose(p1.grad, p2.grad, rtol=1e-3, atol=1e-2)
clear_ctx_dicts(feature_model2)